In [43]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)

In [44]:
import pandas as pd

tabela =pd.read_excel('Processos.xlsx')


In [45]:
from selenium.webdriver import ActionChains
import time

for i, linha in enumerate(tabela.index):
  # Pegar o html dos meus arquivos
  driver.get('file:///home/albert/project/python/curso_py/08_automacao_web/01/index.html')

  # Encontrar o botão que me da as opções que me permite selecionar um estado
  button = driver.find_element(By.XPATH, '/html/body/div/div/button')
  ActionChains(driver).move_to_element(button).perform()

  #Seleciono um estado
  cidade = tabela.loc[linha, "Cidade"]
  driver.find_element(By.PARTIAL_LINK_TEXT, cidade).click()

  # Troco o selenuim para outro tab
  aba_original = driver.window_handles[0]
  indice = 1 + linha
  nova_aba =driver.window_handles[indice]
  driver.switch_to.window(nova_aba)

  # Preencho o formulario
  driver.find_element(By.XPATH, '/html/body/div/form/div/input[1]').send_keys(tabela.loc[linha, "Nome"])
  driver.find_element(By.XPATH, '/html/body/div/form/div/input[2]').send_keys(tabela.loc[linha, "Advogado"])
  driver.find_element(By.XPATH, '/html/body/div/form/div/input[3]').send_keys(str(tabela.loc[linha, "Processo"]))
  driver.find_element(By.CLASS_NAME, 'registerbtn').click()

  # Aceito o alerta
  alert =driver.switch_to.alert
  alert.accept()

  # Aceito o segundo alarme e lanço uma mensagem dependendo do que me retornam
  while True:
    try:
      alert = driver.switch_to.alert
      break
    except:
      time.sleep(1)

  # Trocar os status no excel
  txt_alert = alert.text
  if 'Processo encontrado com sucesso' in txt_alert:
    alert.accept()
    tabela.loc[linha, 'Status'] = "Encontrado"
  else:
    tabela.loc[linha, 'Status'] = "Não encontrado"
    alert.accept()


/tmp/ipykernel_123125/443667637.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Encontrado' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tabela.loc[linha, 'Status'] = "Encontrado"


In [46]:
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Não encontrado
3,Carol,Amanda,PC5197,São Paulo,Encontrado
